In [2]:
import os
import numpy as np
import pandas as pd
import random 
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import time


# reading FASTQs from illumina

# =============================================================================

def find_nth_n_str(string, leter, n):
    
    """ finds index of the nth letter in a string 
    
    find_nth_n_str(string, leter,n)
        string: str to search 
        leter: str charater to find 
        n: int how many letters to skip-1
        
    """
    ################### check inputs #########################
    if type(string) != str:
        raise TypeError('string must be of type str not '+ str(type(string)))
    if type(leter) != str:
        raise TypeError('leter must be of type str not '+ str(type(leter)))
    if type(n) != int:
        raise TypeError('n must be of type str not '+ str(type(n)))
    ##########################################################
        
    m=0
    for c, char in enumerate(string):
        if m<n:
            if char == leter:
                m+=1
            else:
                pass
        else:
            return(c-1)

# =============================================================================

def read_illlumina_FASTQ(FASTQ_path):
    
    """ read illumina fastq and return line ID, sequence, and Q score
    
    read_illlumina_FASTQ(FASTQ_path)
        FASTQ_path: fastq path
        
    """
    
    fastq_file = open(os.path.normpath(FASTQ_path), 'r')
    ID_SEQ_Q = {}
    line_count = 0

    while True:
        line = fastq_file.readline()
        line_count += 1
        if line == '':
            continue
        if line == 'end':
            break
        if line[0:2] == '@M':
            ID_line =  line
            seq_line = fastq_file.readline().replace('\n','')
            line_count += 1
            Plus_line = fastq_file.readline().replace('\n','')
            line_count += 1
            quality_line = fastq_file.readline().replace('\n','')
            line_count += 1
            Q = np.mean([ord(q)-33 for q in quality_line])
            #@M00179:299:000000000-KDNHP:1:1101:9576:1149 1:N:0:NNNNNNNN+NNNNNNCN
            #@<instrument>:<run number>:<flowcell ID>:<lane>:<tile>:<x-pos>:<y-pos>:<UMI> <read>:<is filtered>:<control number>:<index>
            ID = ID_line[:find_nth_n_str(ID_line,' ',1)]
            ID_SEQ_Q[ID] = (seq_line,Q)
    fastq_file.close()
    print(str(line_count/4))
    return(ID_SEQ_Q)

# =============================================================================

def comR1_R2(FASTQ_R1,FASTQ_R2):
    
    """ read illumina fastqs and create combind datafram for read 1 and 2
    
    comR1_R2(FASTQ_R1,FASTQ_R2)
        FASTQ_R1: fastq read 1 path 
        FASTQ_R2: fastq read 2 path 
        
    """
    
    Read1 = read_illlumina_FASTQ(FASTQ_R1)
    print('fastQ 1 red')
    Read2 = read_illlumina_FASTQ(FASTQ_R2)
    print('fastQ 2 red')
    
    R1 = []
    R2 = []
    Q1 = []
    Q2 = []
    readnum = 0
    for Id in Read1:
        if Read2.get(Id) == None:
            continue
        else:
            r2, q2 = Read2[Id]
            r1, q1 = Read1[Id]
            R1.append(r1)
            R2.append(r2)
            Q1.append(q1)
            Q2.append(q2)
            readnum += 1
    df = pd.DataFrame(data = {'R1':R1,'R2':R2,'Q1':Q1,'Q2':Q2}, columns = ['R1','R2','Q1','Q2'])  
    return(df)

# =============================================================================
        

In [ ]:
# look up tables (dict) for analysis

# =============================================================================

def DNA2int(DNA, N = 'A'):
    ''' Converts DNA into intiger form.
    
        DNA: str DNA with bases A,T,G,C,a,t,g,c 
        N: 'random', None, or bases A,T,G,C,a,t,g,c 
        returns an intager repersentation of DNA, if DNA has base not in above 
        list, return 0
        
    '''    
    d2b = {'A':'00','C':'01','T':'10','G':'11','a':'00','c':'01','t':'10','g':'11'}
    bs = '1'
    # binary 000000 -> 0 not AAAA so all codes have to start with a 1 so no ambugiuity
    for base in DNA:
        if d2b.get(base) == None:
            if N != None and base in 'Nn':
                if N == 'random':
                    base = random.choice(['A','T','C','G'])
                elif type(N) == str:
                    base = N
            else:
                raise Exception('base <'+base+'> not in libuary (A,T,G,C,a,t,g,c)') 
        bs+=d2b[base]
    intager = int(bs, 2)
    return(intager)

# =============================================================================

def int2DNA(intager):
    ''' Converts intager into DNA form.
    
        DNA: int DNA 
        returns an DNA str with bases A,T,G,C or None if intager is 0
        
    '''
    b2d = {'00':'A','01':'C','10':'T','11':'G'}
    # 3 because the bin finction gives '0b1(the code for seq)' the leading 1 is because leading 0 are ambiguous 
    bi = bin(intager)[3:]
    seq = ''
    for sec in range(int(len(bi)/2)):
        loc = sec*2
        seq += b2d[bi[loc:loc+2]]
    return(seq)
    
# =============================================================================

def hamming_close_1(seq):
    
    ''' return DNA sequences list that are hamming distance 1 away 
    
        seq: str DNA sequence with bases A,T,G,C
        
    '''
    
    close = []
    seq = list(seq)
    alph = ['A','C','T','G']
    for base in range(len(seq)):
        for N in alph:
            N_seq = seq[0:base] +[N] + seq[base+1:]
            close.append("".join(N_seq))
    return(close)

# =============================================================================

def hamming_close(seq, distance = 1):
    ''' return DNA sequences list that are hamming distance away definded by distance
    
        seq: str DNA sequence with bases A,T,G,C
        distance: int, max hamming distance of sequences returned
        
    '''
    h0 = [seq]
    hammings_done = 0
    while True:
        if hammings_done == distance:
            break
        else:
            hp = []
            for w in h0:
                hp += hamming_close_1(w)
            h0 = hp
            hammings_done += 1
    return(h0)

# =============================================================================

def hamming_close_FP(seq,FP):
    ''' return DNA sequences list that are hamming distance 2 away and list of name FP that is 
        same length as returned list of sequences
    
        seq: str DNA sequence with bases A,T,G,C
        FP: str, name of sequense to be retuned in list
        
    '''
    
    HC_seqs = [DNA2int(DNA, N = 'A') for DNA in hamming_close(seq,distance = 2)]
    FPs = [FP]*len(HC_seqs)
    return(HC_seqs,FPs)

# =============================================================================

# probe lookup table 

Gene_probe_path = 'Path/to/CSV' #formated CSV with columns named: gene_sym, targeting_seqs
# with the targeting seq formated like "ACTGACTG, ACTGACTG". -> left probe, right probe

Gene_probe = pd.read_csv(Gene_probe_path)

# number probes if not numbered to make unique names
Gene_probe_path_lib = {}
name_dict = {}
for FP,seq in zip(Gene_probe['gene_sym'].tolist(),
                  Gene_probe['targeting_seqs'].tolist()):
    
    num=1
    while name_dict.get(FP+'_'+str(num)) != None:
        num+=1
    
    num_name = FP+'_'+str(num)
    name_dict[FP+'_'+str(num)]=num
    
    Gene_probe_path_lib[seq[-30:]] = [num_name,FP,'R']
    Gene_probe_path_lib[seq[:30]] = [num_name,FP,'L']

# generate probe lookup table
probe_Look_up = {} 

SEQ = []
FP_num = []
for seq in Gene_probe_path_lib:
    FP_num.append(Gene_probe_path_lib[seq][0])
    SEQ.append(seq)

print('calulating HC probes')

threads = 32 # number of cores to run calulaiton on
pool = ThreadPool(threads)
results = pool.starmap(hamming_close_FP, zip(SEQ,FP_num),20)      
pool.close() 
pool.join()

for result in results:
    for key, value in zip(result[0],result[1]):
        probe_Look_up[key] = value 
                
    
# =============================================================================

#for round 1 and 2 barcodes
BClookUPpath = 'Path/to/CSV' #formated CSV with columns named: 'Well Position', 'Name'
# with the 'Well Position' being the well of a 96welll plat and 'Name' being the DNA sequence of the barcode


#for round 3 barcodes
BClookUPpath3 = 'Path/to/CSV' #formated CSV with columns named: 'Well Position', 'Name'
# with the 'Well Position' being the well of a 96welll plat and 'Name' being the DNA sequence of the barcode

lookupDF = pd.read_csv(BClookUPpath)
lookupDF3 = pd.read_csv(BClookUPpath3)
BC_lookupR3pre = {}
BC_lookupR3_1 = {}

for well, seq in zip(lookupDF3['Well Position'].tolist(),lookupDF3['Name'].tolist()):
        seq = reverse_complement(seq)
        BC_lookupR3_1[seq] = well 
        for HC in hamming_close(seq,distance = 2):
            if BC_lookupR3pre.get(HC) == None: 
                BC_lookupR3pre[HC] = []
            BC_lookupR3pre[HC].append(well)
     
BC_lookupR3 = {}

# remove ambigious BCs
for seq in BC_lookupR3pre:
    if len(set(BC_lookupR3pre[seq])) == 1:
        BC_lookupR3[seq] = BC_lookupR3pre[seq][0]   


for well, seq in zip(lookupDF['Well Position'].tolist(),lookupDF['Name'].tolist()):
    BC_lookupR1_R2_1[seq] = well 
    for HC in hamming_close(seq,distance = 2):
        if BC_lookupR1_R2pre.get(HC) == None: 
            BC_lookupR1_R2pre[HC] = []
        BC_lookupR1_R2pre[HC].append(well)
     
BC_lookupR1_R2 = {}

# remove ambigious BCs
for seq in BC_lookupR1_R2pre:
    if len(set(BC_lookupR1_R2pre[seq])) == 1:
        BC_lookupR1_R2[seq] = BC_lookupR1_R2pre[seq][0]



        

In [ ]:
# analysis of sequences for hybriseq

# =============================================================================

def getKmer(seq, k):
    
    ''' get all subsequences from seq of length k return list 
    
        getKmer(seq, k)
            seq: str 
            k: int subseq to lenth to find
    '''
    
    if k>len(seq):
         return([])
    # catch case len seq = k
    if len(seq)==k:
        return([seq])
    kmer = []
    for i in range(len(seq)-k):
        kmer.append(seq[i:i+k])
    return(kmer)

# =============================================================================


# look for BC, exact first then close, args: close, exact 
def lookUPalg(BC_lookup,
              BC_lookup_1,
              seq_line,
              start = -9,
              stop = 0,
              k = 7):
  
    kmer = getKmer(seq_line[start:stop], k = k)
    exact = []
    exactP = []
    close = []
    closeP = []
    for sub in kmer:
        if BC_lookup_1.get(sub)!= None:
            exact.append(BC_lookup_1.get(sub))
            
            exactP.append(seq_line.find(sub,start,stop))
    if len(exact) != 0:
        return(exact,close,exactP,closeP)
    
    else:
        for sub in kmer:
            if BC_lookup.get(sub)!= None:
                close.append(BC_lookup.get(sub))
                closeP.append(seq_line.find(sub,start,stop))
    if len(close) != 0:
        return(exact,close,exactP,closeP)
    return(None)

# lookuptable is intager not DNA
def probe_LU_slide2(subseq, lookuptable,simdistance =30):
    Fp_inSeq = []
    for sub in [DNA2int(DNA, N = 'A') for DNA in getKmer(subseq,simdistance)]:
        if lookuptable.get(sub)!= None:
            Fp_inSeq.append(lookuptable[sub])    
    if len(set(Fp_inSeq)) >= 1:
        return(Fp_inSeq[0],subseq)
    else:
        return('none', 'none')

def read_DFs(RAWreadDF, # dataframe with raw reads 
             BC_lookupR3, #look up dict for barcode sequences round 1&2 hamming = 1
             BC_lookupR3_1,# look up dict for barcode sequences round 3 hamming = 0 
             BC_lookupR1_R2, # look up dict for barcode sequences round 1&2 hamming = 1 
             BC_lookupR1_R2_1, # look up dict for barcode sequences round 1&2 hamming = 0
             probe_Look_up,# look up dict for probe sequences hamming = 2
             simdistance = 30, # probe length
             threads = 16):  # number of cores to run analysis on 
    
    '''
        read raw reads datafram and return datafram with sequence info
    
    '''
    
    
    t0  = time.time()
    total_reads = 0
        
    # extract sequence info
    def  read_seq_line(line,
                       line2,
                       BC_lookupR3,
                       BC_lookupR3_1,
                       BC_lookupR1_R2,
                       BC_lookupR1_R2_1,
                       probe_Look_up,
                       simdistance):

        seq_line = line
        seq_line2 = line2

        seq_line = reverse_complement(seq_line)
        
        all_reads1 = seq_line
        all_reads2 = seq_line2
    
        #read 1 stuff
        R3 = 0
        R2 = 0
        R1 = 0
        
        ## check for specfic regions in R1, R2, R3 
        R1_offset = 0
        R2_offset = 35
        R3_offset = 57
        
        R1_start = seq_line[R1_offset:25].find('ATTCG')
        R2_start = seq_line[R2_offset:56].find('TGCTTGAG')
        R3_start = seq_line[R3_offset:].find('GTTTCG')

        # check if all regions are there
        if R3_start != -1:
            R3 = 1 
        if R2_start != -1:
            R2 = 1       
        if R1_start != -1:
            R1 = 1
        total = R3 + R2 + R1 
        if total != 3:
            fa = []
            if R3_start == -1:
                fa.append('R3')
            if R2_start == -1:
                fa.append('R2')
            if R1_start == -1:
                fa.append('R1')
            Fail = ''.join(fa)
        else:
            Fail = 'good'
                            
        #if True:
        #R1
        R1_seq = None
        if R1_start != -1:
            R1_s = R1_offset+R1_start+len('ATTCG')
    
            R1_seq = lookUPalg(BC_lookupR1_R2,
                        BC_lookupR1_R2_1,
                        seq_line = seq_line,
                        start = R1_s,
                        stop = R1_s + 7 + 2,
                        k = 7)
        
        #R2
        R2_seq = None

        if R2_start != -1:
            R2_s = R2_offset+R2_start+len('TGCTTGA')

            R2_seq = lookUPalg(BC_lookupR1_R2,
                        BC_lookupR1_R2_1,
                        seq_line = seq_line,
                        start = R2_s,
                        stop = R2_s + 7 + 2,
                        k = 7)
                
        #R3 
        R3_seq = None
        seq_line_len = len(seq_line)
        
        if R3_start != -1:
            R3_s =  R3_offset + R3_start+ len('GTTTCG')
            R3_seq = lookUPalg(BC_lookupR3,
                               BC_lookupR3_1,
                               seq_line = seq_line,
                               start = R3_s,
                               stop = R3_s+7,
                               k = 7)  
        
        #R1
        if R1_seq == None:
            R1_seq = 'none'
            BC1p = 'none'
        else:
            exact = R1_seq[0]
            colose = R1_seq[1]
            exactp = R1_seq[2]
            colosep = R1_seq[3]
            
            #check for ambugious identification 
            if len(exact) == 1:
                R1_seq = exact[0]
                BC1p = exactp[0]
            elif len(colose) == 1:
                R1_seq = colose[0]
                BC1p = colosep[0]
            else:
                R1_seq = 'none'    
                BC1p = 'none'
        
        #R2
        if R2_seq == None:
            R2_seq = 'none'
            BC2p = 'none'
        else:
            exact = R2_seq[0]
            colose = R2_seq[1]
            exactp = R2_seq[2]
            colosep = R2_seq[3]
            
            #check for ambugious identification 
            if len(exact) == 1:
                R2_seq = exact[0]
                BC2p = exactp[0]
            elif len(colose) == 1:
                R2_seq = colose[0]
                BC2p = colosep[0]
            else:
                R2_seq = 'none'    
                BC2p = 'none'
        
        #R3       
        if R3_seq == None:
            R3_seq = 'none'
            BC3p = 'none'
        else:
            exact = R3_seq[0]
            colose = R3_seq[1]
            exactp = R3_seq[2]
            colosep = R3_seq[3]
            
            #check for ambugious identification 
            if len(exact) == 1:
                R3_seq = exact[0]
                BC3p = exactp[0]
            elif len(colose) == 1:
                R3_seq = colose[0]
                BC3p = colosep[0]
            else:
                R3_seq = 'none'    
                BC3p = 'none'
            
        BC1 = R1_seq
        BC2 = R2_seq
        BC3 = R3_seq
    
        UMI = seq_line2[60:68]

        found = 'none'

        ID_R, rseq = probe_LU_slide2(subseq = seq_line2[:30],
                                     lookuptable = probe_Look_up,
                                     simdistance = simdistance)
        
        ID_L, lseq = probe_LU_slide2(subseq = seq_line2[30:60],
                                     lookuptable = probe_Look_up,
                                     simdistance = simdistance)
                    
        LeftID = ID_L
        if lseq == 'none':
            LeftSeq = 'none'
            Left_probe_num = 'none'
        else:
            LeftSeq = lseq
            Left_probe_num = ID_L
        
        RightID = ID_R
        if rseq == 'none':
            RightSeq = 'none'
            Right_probe_num = 'none'
        else:
            RightSeq = rseq
            Right_probe_num = ID_R
            
        return(all_reads1,
               all_reads2,
               Fail,
               BC1,
               BC2,
               BC3,
               BC1p,
               BC2p,
               BC3p,
               UMI,
               LeftID,
               RightID,
               LeftSeq,
               RightSeq,
               Left_probe_num,
               Right_probe_num)
                
    # run calulations on mulitiple cores 
    file_num = len(RAWreadDF['R1'])

    # set up threads 
    if threads == -1: 
        threads = multiprocessing.cpu_count()
    pool = ThreadPool(threads)
    
    print('runing multi, CPU = '+str(threads))
    
    
    results = pool.starmap(read_seq_line, zip(RAWreadDF['R1'],
                                              RAWreadDF['R2'],
                                              [BC_lookupR3 for x in range(file_num)],
                                              [BC_lookupR3_1 for x in range(file_num)],
                                              [BC_lookupR1_R2 for x in range(file_num)],
                                              [BC_lookupR1_R2_1 for x in range(file_num)],
                                              [probe_Look_up for x in range(file_num)],
                                              [simdistance for x in range(file_num)]),
                                              20)      
    pool.close() 
    pool.join()
    
    all_reads1_ = [line[0] for line in results]
    all_reads2_ = [line[1] for line in results]
    Fail_ = [line[2] for line in results]
    BC1_ = [line[3] for line in results]
    BC2_ = [line[4] for line in results]
    BC3_ = [line[5] for line in results]
    BC1p_ = [line[6] for line in results]
    BC2p_ = [line[7] for line in results]
    BC3p_ = [line[8] for line in results]

    UMI_ = [line[9] for line in results]
    LeftID_ = [line[10] for line in results]
    RightID_ = [line[11] for line in results]
    LeftSeq_ = [line[12] for line in results]
    RightSeq_ = [line[13] for line in results]
    Left_probe_num_ = [line[14] for line in results]
    Right_probe_num_ = [line[15] for line in results]
          
    read_data = {'all_reads1':all_reads1_ ,
                 'all_reads2':all_reads2_ ,
                 'Fail':Fail_,
                 'BC1':BC1_,
                 'BC2':BC2_,
                 'BC3':BC3_,
                 'BC1p':BC1p_,
                 'BC2p':BC2p_,
                 'BC3p':BC3p_,
                 'UMI':UMI_,
                 'LeftID':LeftID_,
                 'RightID':RightID_,
                 'LeftSeq':LeftSeq_, 
                 'RightSeq':RightSeq_,
                 'Left_num':Left_probe_num_,
                 'Right_num':Right_probe_num_}

    read_col = ['all_reads1' ,
             'all_reads2' ,
             'Fail',
             'BC1',
             'BC2',
             'BC3',
             'BC1p',
             'BC2p',
             'BC3p',
             'UMI',
             'LeftID',
             'RightID',
             'LeftSeq',
             'RightSeq',
             'Left_num',
             'Right_num']
    
    readDF = pd.DataFrame(data = read_data, columns = read_col)
    # return dataframe with reads and regions detected for probe, R1,R2,R3, UMI
    return(readDF)

# filter for valid reads 
def good_reads(readDF):
    '''
        filter datafram for good reads 
    
    '''
    
    readDF['LeqR'] = [L==R for L,R in zip(readDF['Left_num'],readDF['Right_num'])]
    readDF1 = readDF[readDF['LeqR']==True] # ensure same probes for both left and right 
    readDF1 = readDF1[readDF1['Fail']=='good'] # ensure sequence has universal reagions in R1,R2,R3
    readDF1 = readDF1[readDF1['BC1']!='none'] # ensure has barcode 1
    readDF1 = readDF1[readDF1['BC2']!='none'] # ensure has barcode 2
    readDF1 = readDF1[readDF1['BC3']!='none'] # ensure has barcode 3
    readDF1 = readDF1[readDF1['LeftID']!='none'] # ensure has left probe
    readDF1 = readDF1[readDF1['RightID']!='none'] # ensure has right probe
    readDF1['CellID'] = readDF1['BC1']+readDF1['BC2']+readDF1['BC3'] # make cell barcode
    readDF1['UMI_CellID'] = readDF1['UMI']+readDF1['CellID']+readDF1['Left_num'] # make composit UMI
    readDF1.drop_duplicates(subset='UMI_CellID', keep='first', inplace=True) # drop UMI
    return(readDF1)
